# PreProcess csv files from CellProfiler

We built a pipeline to segment red blood cells (RBC), nuclei (based on hematoxylin), and nucleated cells (based on eosin).  
* CP_20220429f = Ypos  
* CP_20220429g = Yneg  

We used MeasureShape and MeasureNeighbor. Thus, we get about 68 measures per object (spreadsheet columns), for 4 object classes (csv files), for as many objects as were detected in each path (spreadsheet rows).  

    39616 Cells.csv 68 columns
    39616 Nuclei.csv 68 columns
    60887 RBC.csv 60 columns
    12980 Image.csv 62 columns
    153099 total

Here, each image is actually a patch. The image data includes redundant fields: #RBC, #nuclei, #cells. But the rest of the image data seems non-redundant, including thresholds used and total areas covered. CellProfiler assigned image numbers in order starting at 1 for the first patch. 

The Images.csv contains fields called Filename and URL that contain the path and filename of the patch. For example, D:Martinez/B3.2.jpg is patch 2 of tumor B3.

We ran CellProfiler on the full training set (excludes a 20% test set) of center patches. Here, transform per-object metrics into per-patch metrics. Also, exclude patches with no nuclei.

In [1]:
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn
print('sklearn',sklearn.__version__)

Python 3.8.10
sklearn 1.0.2


In [2]:
YPOS_DIR='/home/jrm/Martinez/CellProfilerRuns/CP_20220429f/'
YNEG_DIR='/home/jrm/Martinez/CellProfilerRuns/CP_20220429g/'
IMAGES="Image.csv"
REDS="RBC.csv"
NUKES="Nuclei.csv"
CELLS="Cells.csv"
MIN_NUCLEI = 2

In [3]:
def describe_all_columns(df):
    count1 = df.isnull().sum().sum()
    print('Zero out this many NaN:', count1)
    with pd.option_context('display.max_columns', None):
        print(df.describe(include='all'))
def make_dataframe(filename):
    df = pd.read_csv(filename) 
    return df
def shave_images(df):
    # drop uninformative columns (mostly just zero)
    bad_columns = ['ProcessingStatus', 'Height_HE', 'Width_HE', 'Scaling_HE']
    bad_columns += ['Series_HE', 'Frame_HE', 'Channel_HE', 'Group_Number']
    df = df.drop(columns=bad_columns)
    # drop strings, like filepaths, that could leak the Ypos/Yneg labels
    df = df.select_dtypes(['number'])
    # drop CellProfiler timing, errors (should all be zero), and metadata stats
    df = df.drop(df.filter(regex='^Metadata_').columns, axis=1)
    df = df.drop(df.filter(regex='^ExecutionTime_').columns, axis=1)
    df = df.drop(df.filter(regex='^ModuleError_').columns, axis=1)
    df = df[df['Count_Nuclei']>=MIN_NUCLEI]
    df = df.fillna(0)
    return df


In [4]:
filename = YPOS_DIR+IMAGES
df_ipos = make_dataframe(filename) 
df_ipos = shave_images(df_ipos)
#describe_all_columns(df_ipos)
df_ipos.describe()

,Count_Cells,Count_Nuclei,Count_RBC,Group_Index,ImageNumber,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_FinalThreshold_RBC,Threshold_GuideThreshold_Cells,Threshold_GuideThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_OrigThreshold_RBC,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_SumOfEntropies_RBC,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei,Threshold_WeightedVariance_RBC
count,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000
mean,3.699343,3.699343,4.346731,6446.005098,6446.005098,0.198198,0.229849,0.447377,0.190327,0.231883,0.200201,0.229477,0.372814,-11.732075,-11.436286,-10.138482,0.948547,0.679775,2.199110
std,1.478569,1.478569,3.673069,3889.139278,3889.139278,0.070158,0.051438,0.090652,0.069516,0.057026,0.070901,0.050200,0.075544,0.797363,0.926382,0.882962,0.462473,0.577341,1.485949
min,2.000000,2.000000,0.000000,1.000000,1.000000,0.000166,0.026288,0.000304,0.000165,0.022708,0.000166,0.024872,0.000254,-13.905191,-13.968753,-13.900173,0.008287,0.002667,0.034349
25%,2.000000,2.000000,2.000000,2878.000000,2878.000000,0.162104,0.196336,0.406246,0.153097,0.192319,0.165786,0.196755,0.338538,-12.323464,-11.899810,-10.604216,0.594257,0.285706,1.021031
50%,3.000000,3.000000,4.000000,6650.000000,6650.000000,0.197002,0.234884,0.452503,0.189919,0.242477,0.199161,0.232378,0.377086,-11.903494,-11.494668,-10.201425,0.948817,0.487546,1.873390
75%,5.000000,5.000000,6.000000,9924.000000,9924.000000,0.234956,0.267813,0.498460,0.226771,0.275309,0.237304,0.266793,0.415383,-11.255853,-11.144000,-9.724634,1.288512,0.880290,3.134938
max,10.000000,10.000000,41.000000,12979.000000,12979.000000,0.513549,0.590891,0.707806,0.526853,0.554902,0.513549,0.601256,0.589838,-7.785976,-5.882243,0.000000,2.795326,3.681442,9.340085


In [5]:
filename = YNEG_DIR+IMAGES
df_ineg = make_dataframe(filename) 
df_ineg = shave_images(df_ineg)
#describe_all_columns(df_ineg)
df_ineg.describe()

,Count_Cells,Count_Nuclei,Count_RBC,Group_Index,ImageNumber,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_FinalThreshold_RBC,Threshold_GuideThreshold_Cells,Threshold_GuideThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_OrigThreshold_RBC,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_SumOfEntropies_RBC,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei,Threshold_WeightedVariance_RBC
count,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000,14873.000000
mean,3.957036,3.957036,3.508035,9014.261615,9014.261615,0.193183,0.232311,0.414436,0.186639,0.234816,0.194638,0.231926,0.345364,-11.506303,-11.281784,-10.010848,0.787749,0.495452,1.955225
std,1.579725,1.579725,3.519821,5181.611479,5181.611479,0.079657,0.043542,0.096308,0.081130,0.048858,0.079618,0.042573,0.080257,0.917771,0.952213,1.189379,0.474008,0.463761,1.706803
min,2.000000,2.000000,0.000000,1.000000,1.000000,0.000250,0.020551,0.000368,0.000269,0.020493,0.000250,0.020545,0.000307,-13.900045,-14.217982,-13.879088,0.004965,0.002697,0.010274
25%,3.000000,3.000000,1.000000,4341.000000,4341.000000,0.144394,0.208142,0.379074,0.133939,0.208240,0.149888,0.207586,0.315895,-12.263990,-11.728599,-10.557552,0.385899,0.210314,0.669665
50%,4.000000,4.000000,3.000000,9133.000000,9133.000000,0.195527,0.240153,0.428010,0.186990,0.247547,0.197753,0.238625,0.356675,-11.662491,-11.392960,-10.057407,0.746173,0.354169,1.406422
75%,5.000000,5.000000,5.000000,13642.000000,13642.000000,0.247884,0.263945,0.471838,0.243635,0.270187,0.248085,0.263287,0.393198,-10.875579,-11.022301,-9.577021,1.140852,0.612824,2.778047
max,12.000000,12.000000,42.000000,17912.000000,17912.000000,0.498625,0.394072,0.684108,0.504537,0.402519,0.502132,0.394072,0.570090,-7.532630,-5.834748,0.000000,2.993972,3.754664,10.829500


Aggregator [tutorial](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/).
Python built-in functions [list](https://docs.python.org/3/library/functions.html).


In [6]:
def shave_reds(df):
    num_cols = df.columns.values.tolist()
    num_cols.remove('ImageNumber')
    num_cols.remove('ObjectNumber')
    num_cols.remove('Location_Center_X')
    num_cols.remove('Location_Center_Y')
    num_cols.remove('Location_Center_Z')
    num_cols.remove('Number_Object_Number')      
    agg_cols = {}
    for col in num_cols:
        agg_cols[col]=[min,max,np.mean,np.std]
    agg_cols['ObjectNumber']=[max]
    df = df[df['ImageNumber']>0].groupby('ImageNumber').agg(agg_cols)
    df = df.fillna(0)
    return df

In [7]:
filename = YPOS_DIR+REDS
df_rpos = make_dataframe(filename) 
df_rpos = shave_reds(df_rpos)
df_rpos.head()

AreaShape_Area                                \
                       min   max        mean         std   
ImageNumber                                                
1                      335   587  405.800000  102.726335   
2                      368   609  465.857143  103.246354   
3                      360  1214  670.285714  325.690914   
4                      432   508  470.000000   53.740115   
5                      389  1438  908.166667  459.363654   

            AreaShape_BoundingBoxArea                                 \
                                  min   max         mean         std   
ImageNumber                                                            
1                                 533  1178   766.400000  278.347445   
2                                 616   990   830.571429  157.656649   
3                                 558  2920  1523.571429  893.488268   
4                                 696  1075   885.500000  267.993470   
5                                 672  3038  1625.666667  941.521676   

            AreaShape_BoundingBoxMaximum_X       ... AreaShape_Zernike_9_5  \
                                       min  max  ...                   std   
ImageNumber                                      ...                         
1                                       28  196  ...              0.002340   
2                                       54  203  ...              0.002460   
3                                       34  224  ...              0.003319   
4                                      103  178  ...              0.000935   
5                                       45  224  ...              0.001545   

            AreaShape_Zernike_9_7                                \
                              min       max      mean       std   
ImageNumber                                                       
1                        0.004830  0.011568  0.007626  0.002476   
2                        0.002937  0.011398  0.005770  0.002833   
3                        0.003413  0.019292  0.008709  0.005355   
4                        0.001533  0.005840  0.003686  0.003046   
5                        0.000291  0.013358  0.005866  0.004328   

            AreaShape_Zernike_9_9                               ObjectNumber  
                              min       max      mean       std          max  
ImageNumber                                                                   
1                        0.001434  0.004422  0.002700  0.001349            5  
2                        0.001007  0.014364  0.006812  0.004692            7  
3                        0.002975  0.009223  0.005838  0.002058            7  
4                        0.000358  0.005038  0.002698  0.003309            2  
5                        0.000613  0.007169  0.003813  0.002628            6  

[5 rows x 221 columns]

In [8]:
filename = YNEG_DIR+REDS
df_rneg = make_dataframe(filename) 
df_rneg = shave_reds(df_rneg)
df_rneg.head()

AreaShape_Area                                \
                       min   max        mean         std   
ImageNumber                                                
1                      326   326  326.000000    0.000000   
2                      656   656  656.000000    0.000000   
3                      321  1373  850.666667  526.038338   
4                      384   957  622.000000  298.561551   
5                      442   921  643.000000  210.287105   

            AreaShape_BoundingBoxArea                                  \
                                  min   max         mean          std   
ImageNumber                                                             
1                                 444   444   444.000000     0.000000   
2                                1170  1170  1170.000000     0.000000   
3                                 432  3015  1649.000000  1297.930275   
4                                 720  2268  1313.333333   834.863662   
5                                 756  1440  1063.500000   338.413455   

            AreaShape_BoundingBoxMaximum_X       ... AreaShape_Zernike_9_5  \
                                       min  max  ...                   std   
ImageNumber                                      ...                         
1                                      224  224  ...              0.000000   
2                                      213  213  ...              0.000000   
3                                       12  101  ...              0.002821   
4                                       18  113  ...              0.006109   
5                                       86  223  ...              0.003745   

            AreaShape_Zernike_9_7                                \
                              min       max      mean       std   
ImageNumber                                                       
1                        0.005488  0.005488  0.005488  0.000000   
2                        0.007225  0.007225  0.007225  0.000000   
3                        0.002655  0.007320  0.004739  0.002372   
4                        0.006217  0.007172  0.006749  0.000486   
5                        0.003436  0.011464  0.006879  0.003397   

            AreaShape_Zernike_9_9                               ObjectNumber  
                              min       max      mean       std          max  
ImageNumber                                                                   
1                        0.006943  0.006943  0.006943  0.000000            1  
2                        0.003105  0.003105  0.003105  0.000000            1  
3                        0.001502  0.003882  0.002320  0.001353            3  
4                        0.003527  0.004779  0.004243  0.000645            3  
5                        0.002548  0.005582  0.004592  0.001391            4  

[5 rows x 221 columns]

Merge documentation at [pandas](https://pandas.pydata.org/docs/dev/user_guide/merging.html).

